In [4]:
import json
import sqlite3
import pandas as pd
import config
import numpy as np
import requests
import sys

from sqlalchemy import create_engine
engine = create_engine('postgresql://'+config.db['user']+':'+config.db['password']+config.db['url'])

sSource = "api"
if sSource=="file":
    with open('./data/NVDRstock-trading31-01-23.json','r') as f:
        data = json.loads(f.read())
elif sSource=="api":
    r = requests.get('https://www.set.or.th/api/set/nvdr-trade/stock-trading?sortBy=symbol')
    data = r.json()
    if r.status_code!=200:
        exit

#df = pd.json_normalize(r.json(), record_path =['nvdrTradings'])

cdate = pd.DataFrame(data).iloc[0].astype(str).str[:10]['date']
#df.info()
print(cdate)
df = pd.json_normalize(data, record_path =['nvdrTradings'])
df.drop(['buyVolume', 'sellVolume','netVolume', 'totalVolume','underlyingVolume', 'percentVolume'], axis=1, inplace=True)


#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)

2023-02-02
      symbol                    trddate         buy        sell          net  \
0       24CS  2023-02-02T00:00:00+07:00  6095160.00   4683612.0   1411548.00   
1         2S  2023-02-02T00:00:00+07:00       38.74     17680.0    -17641.26   
2        7UP  2023-02-02T00:00:00+07:00  1089564.00    743645.0    345919.00   
3         A5  2023-02-02T00:00:00+07:00  4523350.00    820614.0   3702736.00   
4        AAI  2023-02-02T00:00:00+07:00  4115135.00  17690505.0 -13575370.00   
..       ...                        ...         ...         ...          ...   
739     YONG  2023-02-02T00:00:00+07:00  1726966.00    642684.0   1084282.00   
740    YUASA  2023-02-02T00:00:00+07:00     2810.00         0.0      2810.00   
741      ZEN  2023-02-02T00:00:00+07:00   594450.00    202640.0    391810.00   
742     ZIGA  2023-02-02T00:00:00+07:00  1279374.00   1383804.0   -104430.00   
743  ZIGA-W1  2023-02-02T00:00:00+07:00     8996.00         0.0      8996.00   

           total     trdvalu

In [5]:
sql = "select * from snvdr where trddate = '"+ cdate + "'"
print(sql)
df = df.loc[~((df['buy'] == '-') | (df['sell'] == '-') | (df['net'] == '-'))]
print(df)
rst = pd.read_sql(sql, con=engine)
print(len(rst.axes[0]))
if len(rst.axes[0]) == 0:df.to_sql(name='snvdr', con=engine, if_exists='append')
cur = engine.connect()
cur.execute("DELETE FROM snvdrsum")
cur.execute("INSERT into snvdrsum(Symbol, sumn, counts) select symbol,round(sum(net)) as sumn,count(symbol)counts from snvdr group by symbol")

# sqlAvg = "UPDATE snvdr SET  \
# ft=(SELECT round(abs(n1.net) / (s.value + 1::numeric) / 10::numeric, 1) as tf  \
# FROM snvdr n1 inner join sprice s on n1.symbol = s.series and n1.TrdDate=s.trddate where n1.Symbol = snvdr.symbol AND n1.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0)), \
# calvol=(SELECT round((n1.net) / ((s.open+s.close)/2::numeric), 1) as calvol   \
# FROM snvdr n1 inner join sprice s on n1.symbol = s.series and n1.TrdDate=s.trddate where n1.Symbol = snvdr.symbol AND n1.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0))  \
# where snvdr.TrdDate = (SELECT DISTINCT TrdDate FROM ssfOI ORDER by TrdDate DESC  LIMIT 1 OFFSET 0)"
# print(sqlAvg)
#cur.execute(sqlAvg)

select * from snvdr where trddate = '2023-02-02'
      symbol                    trddate         buy        sell          net  \
0       24CS  2023-02-02T00:00:00+07:00  6095160.00   4683612.0   1411548.00   
1         2S  2023-02-02T00:00:00+07:00       38.74     17680.0    -17641.26   
2        7UP  2023-02-02T00:00:00+07:00  1089564.00    743645.0    345919.00   
3         A5  2023-02-02T00:00:00+07:00  4523350.00    820614.0   3702736.00   
4        AAI  2023-02-02T00:00:00+07:00  4115135.00  17690505.0 -13575370.00   
..       ...                        ...         ...         ...          ...   
739     YONG  2023-02-02T00:00:00+07:00  1726966.00    642684.0   1084282.00   
740    YUASA  2023-02-02T00:00:00+07:00     2810.00         0.0      2810.00   
741      ZEN  2023-02-02T00:00:00+07:00   594450.00    202640.0    391810.00   
742     ZIGA  2023-02-02T00:00:00+07:00  1279374.00   1383804.0   -104430.00   
743  ZIGA-W1  2023-02-02T00:00:00+07:00     8996.00         0.0      89

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 741 entries, 0 to 740
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   symbol    741 non-null    object 
 1   trddate   741 non-null    object 
 2   buy       741 non-null    float64
 3   sell      741 non-null    float64
 4   net       741 non-null    float64
 5   total     741 non-null    float64
 6   trdvalue  741 non-null    float64
 7   percent   741 non-null    float64
 8   ft        741 non-null    float64
dtypes: float64(7), object(2)
memory usage: 57.9+ KB


In [1]:
import pandas as pd
#import datetime
import sqlite3
# Create your connection.
cnx = sqlite3.connect('EquityTrend.db')
df=pd.read_html('https://www.set.or.th/set/nvdrbystock.do?type=value&sort=total&language=en&country=US')
df=df[0]
#df
df.columns = df.columns.droplevel()
df.rename(columns = {'%*':'Percent'}, inplace = True)

In [2]:
#df = df.drop(df.columns[[1,2]],axis='columns')
df

,Symbol,Buy,Sell,Total,Net,Percent
0,KBANK,1098625900.00,678949924.00,1.777576e+09,419675976.00,34.22
1,SCB,743165602.00,385177985.00,1.128344e+09,357987617.00,31.65
2,PTT,512050984.25,549820450.00,1.061871e+09,-37769465.75,28.26
3,SAWAD,594000948.50,454441032.50,1.048442e+09,139559916.00,36.54
4,BANPU,351549741.10,688390906.50,1.039941e+09,-336841165.40,24.90
...,...,...,...,...,...,...
704,U-W4,189.00,-,1.890000e+02,189.00,0.08
705,MPIC,157.00,-,1.570000e+02,157.00,8.19
706,AMARIN,62.50,-,6.250000e+01,62.50,0.06
707,HYDRO-W1,20.00,-,2.000000e+01,20.00,0.00


In [3]:
df['TrdDate'] = pd.to_datetime('today').date()
df['TrdDate'] = df['TrdDate'].apply(pd.DateOffset(-1)).apply(lambda x : x.date())
#df['TrdDate']= df['TrdDate']
df

C:\Users\Guy-k\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\series.py:4138: FutureWarning: DateOffset.__call__ is deprecated and will be removed in a future version.  Use `offset + other` instead.
  mapped = lib.map_infer(values, f, convert=convert_dtype)


,Symbol,Buy,Sell,Total,Net,Percent,TrdDate
0,PTT,1443383383.50,797916024.75,2.241299e+09,6.454674e+08,19.71,2021-06-01
1,HANA,1471033415.75,766770900.00,2.237804e+09,7.042625e+08,28.86,2021-06-01
2,PTTEP,759674900.00,874049524.00,1.633724e+09,-1.143746e+08,42.71,2021-06-01
3,AOT,603554062.50,683704062.50,1.287258e+09,-8.015000e+07,26.55,2021-06-01
4,KCE,730039241.00,513055616.25,1.243095e+09,2.169836e+08,23.84,2021-06-01
...,...,...,...,...,...,...,...
697,SLP,12.96,72.00,8.496000e+01,-5.904000e+01,0.00,2021-06-01
698,HYDRO,-,64.00,6.400000e+01,-6.400000e+01,0.00,2021-06-01
699,TNDT,-,50.50,5.050000e+01,-5.050000e+01,0.00,2021-06-01
700,TFI-W1,20.00,-,2.000000e+01,2.000000e+01,0.01,2021-06-01


In [ ]:
sql = "select * from sNVDR where TrdDate = '"+ df.iloc[0,6].strftime('%Y-%m-%d') + "'"
print(sql)
rst = pd.read_sql(sql, cnx)
print(len(rst.axes[0]))
#if len(rst.axes[0]) == 0:df.to_sql(name='sNVDR', con=cnx, if_exists='append')

In [ ]:
df = df.iloc[0:50]

In [ ]:
df.info()
df

In [ ]:
df.loc[df['Symbol'] == 'TFI']